In [1]:
import os
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [2]:
# Specify the directory path
root = '/gdrive/MyDrive/Irfan/ESC-50-master'
!ls '/gdrive/MyDrive/Irfan/ESC-50-master'

audio			     melspectrograms	    tests
DenseNet201.best.hdf5	     meta		    VGG16.best.hdf5
esc50.gif		     pytest.ini		    VGG19.best.hdf5
InceptionResNetV2.best.hdf5  README.md		    Xception.best.hdf5
InceptionV3.best.hdf5	     requirements.txt
LICENSE			     ResNet152V2.best.hdf5


In [3]:
import sklearn
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score

from keras import metrics, optimizers
from keras.applications.vgg19 import VGG19, preprocess_input
from keras.applications.resnet_v2 import ResNet152V2, preprocess_input
from keras.applications.densenet import DenseNet201, preprocess_input
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input

import tensorflow as tf
from keras.preprocessing import image
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Input
from matplotlib import pyplot as plt
import numpy as np
import json as js

In [4]:
batch_size = 32
epochs = 100

# dimensions of our images.
img_width, img_height = 224, 224

input_tensor = Input(shape=(224,224,3))

nb_training_samples = 1600
nb_validation_samples = 400# Set parameter values

In [5]:
# training generator configuration
training_data_dir = root + '/melspectrograms/training'

training_datagen = image.ImageDataGenerator(
    rescale=1./255)

training_generator = training_datagen.flow_from_directory(
    training_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size)

# validation generator configuration
validation_data_dir = root + '/melspectrograms/testing/'

validation_datagen = image.ImageDataGenerator(
    rescale=1./255)

validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size)

X_train, y_train = next(training_generator)
X_test, y_test = next(validation_generator)

Found 1600 images belonging to 50 classes.
Found 400 images belonging to 50 classes.


In [10]:
def model_maker(model):
  base_model = model(weights='imagenet', include_top=False, input_tensor=input_tensor)

  top_model = Sequential()
  top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
  top_model.add(Dense(256, activation='relu'))
  top_model.add(Dropout(0.5))
  top_model.add(Dense(50, activation='relu')) #CHANGE HERE FOR THE AMOUNT OF CLASS
  top_model.add(Dropout(0.5))
  top_model.add(Dense(2, activation='softmax'))

  output_model = Model(inputs=base_model.input, outputs=top_model(base_model.output))

  output_model.compile(
                optimizer='Adam',
                loss='binary_crossentropy',
                metrics=['accuracy'])
  
  return output_model

In [11]:
VGG19_clf = tf.keras.wrappers.scikit_learn.KerasClassifier(
                            model_maker(VGG19),
                            epochs=epochs,
                            verbose=False)
VGG19_clf._estimator_type = "classifier"
ResNet152V2_clf = tf.keras.wrappers.scikit_learn.KerasClassifier(
                            model_maker(ResNet152V2),
                            epochs=epochs,
                            verbose=False)
ResNet152V2_clf._estimator_type = "classifier"
DenseNet201_clf = tf.keras.wrappers.scikit_learn.KerasClassifier(
                            model_maker(DenseNet201),
                            epochs=epochs,
                            verbose=False)
DenseNet201_clf._estimator_type = "classifier"
InceptionV3_clf = tf.keras.wrappers.scikit_learn.KerasClassifier(
                            model_maker(InceptionV3),
                            epochs=epochs,
                            verbose=False)
InceptionV3_clf._estimator_type = "classifier"
InceptionResNetV2_clf = tf.keras.wrappers.scikit_learn.KerasClassifier(
                            model_maker(InceptionResNetV2),
                            epochs=epochs,
                            verbose=False)
InceptionResNetV2_clf._estimator_type = "classifier"

In [12]:
voting = VotingClassifier(
             estimators=[('VGG19', VGG19_clf),
                         ('ResNet152V2', ResNet152V2_clf),
                         ('DenseNet201', DenseNet201_clf),
                         ('InceptionV3',InceptionV3_clf),
                         ('InceptionResNetV2', InceptionResNetV2_clf)], 
             voting='soft',
             flatten_transform=True)

In [9]:
voting.fit(X_train,y_train)

NotImplementedError: ignored